In [1]:
import pandas as pd

In [2]:
# Load data from advent of code into a dataframe
url = "https://adventofcode.com/2023/day/2/input"
headers = {"Cookie": "session=blah"}
df = pd.read_csv(url, storage_options=headers, sep=": ", names=["game_id", "games"], engine='python')
df.head()

,game_id,games
0,Game 1,"2 green, 6 blue, 7 red; 12 green, 6 blue, 3 re..."
1,Game 2,"10 green, 4 red; 2 red; 12 green, 11 red, 1 bl..."
2,Game 3,"3 green; 15 red, 7 blue, 1 green; 3 red, 6 blu..."
3,Game 4,"5 blue; 8 blue, 7 red; 9 blue, 5 red, 4 green;..."
4,Game 5,"3 blue, 4 red, 10 green; 13 green, 8 blue, 2 r..."


In [3]:
# Convert to series with game_id (as an int) as index
df["game_id"] = df["game_id"].apply(lambda x: int(x.split(" ")[1]))
df = df.set_index("game_id")["games"]
df.head()

game_id
1    2 green, 6 blue, 7 red; 12 green, 6 blue, 3 re...
2    10 green, 4 red; 2 red; 12 green, 11 red, 1 bl...
3    3 green; 15 red, 7 blue, 1 green; 3 red, 6 blu...
4    5 blue; 8 blue, 7 red; 9 blue, 5 red, 4 green;...
5    3 blue, 4 red, 10 green; 13 green, 8 blue, 2 r...
Name: games, dtype: object

In [4]:
# Process the data into long format, with a row per show (multiple shows per game) with the number of blue / green / red shown
df = (
    df
    # Pivot longer by the show number 
    .str.split("; ", expand=True).stack()
    .rename_axis(index={None: "show_num"})
    # Pivot longer by each colour
    .str.split(", ", expand=True).stack()
    .rename_axis(index={None: "pick_num"})
    # Extract number (index 0) and colour (index 1) from pick
    .str.split(" ", expand=True)
    # Ensure the number is an int
    .set_index(1, append=True)
    .astype(int)
    .reset_index(level=1)
    # Pivot wide
    .pivot_table(index=["game_id", "show_num"], values=0, columns=1, aggfunc="sum", fill_value=0)
    .reset_index()
)
df.head()

1,game_id,show_num,blue,green,red
0,1,0,6,2,7
1,1,1,6,12,3
2,1,2,4,18,5
3,2,0,0,10,4
4,2,1,0,0,2


In [5]:
# Part 1 - Sum the unique game ids that are possible
idx_not_possible = (df["blue"] > 14) | (df["green"] > 13) | (df["red"] > 12)
sum(set(df["game_id"]) - set(df.loc[idx_not_possible, "game_id"]))

2076

In [6]:
# Part 2 - find the max of each colour per game, multiply together and sum
df.groupby(["game_id"])[["blue", "green", "red"]].max().prod(axis=1).sum()

70950